In [5]:
!pip install pymorphy2

In [7]:
from selenium import webdriver

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup as bs

from time import sleep

from webdriver_manager.chrome import ChromeDriverManager

import csv

import requests

from lxml import html

import re

import unicodedata

import pymorphy2



In [21]:
with open('lyasdata.txt', 'r', encoding='utf8') as f:
    f = f.read()

In [27]:
topics = f.split('\n')

topics = [topic.split(': ')[-1] for topic in topics if topic != '']

In [28]:
topics

['древний возраст кость геном слой находка поздний след популяция днк сапиенс неандерталец зуб признак генетический',
 'звезда масса галактика чёрный_дыра вселенная излучение солнечный космический наблюдение астроном солнце звёздный спектр ядро дыра',
 'белок ген днк опухоль мутация мышь ткань рак участок вирус бактерия геном фермент белка рецептор',
 'частица энергия физика ядро электрон нейтрино детектор протон измерение масса распад коллаборация событие коллайдер нейтрон',
 'вода температура атмосфера океан период порода содержание слой кислород глубина морской углерод состав зона лёд',
 'самец самка поведение мозг нейрон сигнал особь половой муха запах потомство отбор частота популяция различие',
 'сверхпроводник сверхпроводимость магнитный_поле мантия температура сверхпроводящий магнитный ядро зона материал вихрь образование критический_температура плёнка поле',
 'молекула химический реакция вода атом водород синтез свойство соединение ион материал молекулярный концентрация электр

In [12]:
ACCENT_MAPPING = {
    '́': '',
    '̀': '',
    'а́': 'а',
    'а̀': 'а',
    'е́': 'е',
    'ѐ': 'е',
    'и́': 'и',
    'ѝ': 'и',
    'о́': 'о',
    'о̀': 'о',
    'у́': 'у',
    'у̀': 'у',
    'ы́': 'ы',
    'ы̀': 'ы',
    'э́': 'э',
    'э̀': 'э',
    'ю́': 'ю',
    '̀ю': 'ю',
    'я́́': 'я',
    'я̀': 'я',
}
ACCENT_MAPPING = {unicodedata.normalize('NFKC', i): j for i, j in ACCENT_MAPPING.items()}


def unaccentify(s):
    ## чтобы не было проблем с надстрочными символами
    
    source = unicodedata.normalize('NFKC', s)
    for old, new in ACCENT_MAPPING.items():
        source = source.replace(old, new)
    return source

In [14]:
class googling():
    def __init__(self):
        
        self.driver = webdriver.Chrome(ChromeDriverManager().install())  ##  инициализация веб-драйвера
        self.morph = pymorphy2.MorphAnalyzer()
        
        
        
    def _parse(self):
        ##  внутренняя функция парсинга
        
        soup = bs(self.driver.page_source, 'html.parser')
        search = soup.find_all('div', class_="yuRUbf")
        for h in search:
            self.names.append(h.get_text())


    
    def find(self, word, pages=3):
        ##  ищет первые 30 (по умолчанию) выдач; кол-во страниц можно настраивать
        
        
        self.word = word
        self.names = [] 
        url = "http://www.google.com/search?q=" + self.word
        self.driver.get(url)
        
        self._parse()  ##  парсим
        
        for n in range(pages-1):
            try:
                ##  поиск дополнительных 20 
                ##  смотрим страницу 2 и 3 в поисковой выдаче
                ##  в будущем нужно выделить в отдельную функцию (если будем смотреть все 30 выдач)

                action = ActionChains(self.driver)
                action.key_down(Keys.END).perform()  ##  скролим страницу вниз


                xpath = '//*[@id="pnnext"]/span[2]'
                element = Wait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
                element.click()  ##  нажимаем на "следующий"

                self._parse()  ##  парсим

            except:
                ##  в случае если по этим топикам нет много данных
                pass
        
        self.clean()
        
        
        
    def quit(self):
        ##  функция выхода из веб-драйвера
        ##  по сути дублирует функцию самого драйвера, но в ООП иначе никак
        
        self.driver.quit()
        
        
        
    def _lemmatize(self, text):
        ## лемматизация
        return " ".join([self.morph.parse(word)[0].normal_form for word in text.split()])
        
    
    def clean(self):
        ##  внутренняя функция, призванная очистить от лишних знаков
        ##  надо доработать исходя из того, что требуется в итоге
        
        
        self.result = []

        for name in self.names:
            text = re.split('http', name)[0]
            text = re.sub('[-|a-z|.|,|:|0-9|A-z"]', '', text)
            text = re.sub('  ', ' ', text)
            
            self.result.append(self._lemmatize(text.strip('  ')))
            

    
    def show_topics(self):
        ##  для проверки результатов без сохранения в отдельный файл

        print('QUERY:  ', self.word)
        print('TITLES: \n\n')
        for res in self.result:
            print(res)
        
             
            
    def save(self, kuda):
        ##  сохраняем результаты поисковой выдачи (ОТ ОДНОГО ТОПИКА) в отдельный файл 
        
        source = str(kuda) + '.txt'
        with open(source, 'w', encoding="utf-8") as f:
            f.write(f'QUERY: {self.word}\n')
            f.write('TITLES:\n')
            for res in self.result:
                f.write(res)
                f.write('\n')

In [33]:
yandex_part = googling()



====== WebDriver manager ======
Current google-chrome version is 109.0.5414
Get LATEST chromedriver version for 109.0.5414 google-chrome
Driver [C:\Users\Марк\.wdm\drivers\chromedriver\win32\109.0.5414.74\chromedriver.exe] found in cache
<ipython-input-14-570a913e7587>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(ChromeDriverManager().install())  ##  инициализация веб-драйвера


In [ ]:

##  смотрим топик 5


#yandex_part.find(topics[5])
#yandex_part.show_topics()

In [1]:
## смотрим все 10 топиков, записываем результат от каждого топика в отдельный файл
## ищем 40 выдач для каждого топика (просматриваем 4 страницы)

for n, topic in enumerate(topics):
    yandex_part.find(topic, pages=4)
    yandex_part.save(f'lias_topics/topic_{n}')

NameError: name 'topics' is not defined

In [38]:
yandex_part.quit()

In [1]:
import codecs
import collections
import logging
import re
import pymorphy2
from nltk.tokenize import word_tokenize

logging.basicConfig(level=logging.DEBUG)
morph = pymorphy2.MorphAnalyzer()

C:\conda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\conda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\conda\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
INFO:pymorphy2.opencorpora_dict.wrapper:Loading dictionaries from C:\conda\lib\site-packages\pymorphy2_dicts_ru\data
INFO:pymorphy2.opencorpora_dict.wrapper:format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [2]:
titles = {}

for n in range(0, 10):
    with open(f'lias_topics/topic_{n}.txt', 'r', encoding='utf-8') as f:
        titles[n] = f.read().split('\n')[2:-1]
        
        ##  чтобы было без query & titles добавить [2:]
        ##  пока без лемматизации

In [ ]:
##  Первичная Ранжировка

In [17]:
titles = {}

for n in range(0, 10):
    with open(f'lias_topics/topic_{n}.txt', 'r', encoding='utf-8') as f:
        titles[n] = f.read().split('\n')[2:-1]
        
        ##  чтобы было без query & titles добавить [2:]
        ##  пока без лемматизации

In [18]:
titles[6][:10]

['краткий теория',
 'сверхпроводимость википедия',
 'московский физикотехнический институт',
 'глава физика сверхпроводимость • в гинзбург быть',
 'сверхпроводник',
 'проект российский научный фонд',
 'магнит и сверхпроводник вместе – это сила! наука и жизнь',
 'содержание работа',
 'сверхпроводник википедия',
 'сверхпроводник']

In [19]:
##  сразу ранжируем все выдачи, но! для каждой выдачи по отдельности


query_n = 10  ## the number of filenames (which contain all results from a single query)
topics_dict = {} # {'topic': [word1, word2, ...]}
for number in range(query_n):
    i = 0
    length = len(titles[number]) - 1
    while i < length:
        topics_dict[titles[number][i]] = titles[number][i+1].split(' ')
        i += 2

In [ ]:
##  Переранжировка

In [20]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from nltk import FreqDist

# ПРАВИЛО 1
def one(topics_dict, topic):
    for small_label in topics_dict[topic][::-1]: # идём с конца, потому что если их несколько, тогда лучшие будут в начале
        if len(small_label.split(" ")) == 2:
            for big_label in topics_dict[topic]:
                if len(big_label.split(" ")) > 2:
                    # приводим обе метки в нормальную форму
                    small_label_normalized = []
                    for a_word in small_label.split(" "):
                        small_label_normalized.append(morph.parse(a_word)[0].normal_form)
                    big_label_normalized = []
                    for a_word in big_label.split(" "):
                        big_label_normalized.append(morph.parse(a_word)[0].normal_form)
                    if " ".join(small_label_normalized) in " ".join(big_label_normalized): # если маленькая в большой
                        index_big = topics_dict[topic].index(big_label)
                        del topics_dict[topic][index_big] # удалили длинную
                        index_small = topics_dict[topic].index(small_label)
                        del topics_dict[topic][index_small] # удалили короткую
                        topics_dict[topic].insert(0, small_label) # вставили короткую в начало
    return(topics_dict) 

# ПРАВИЛО 2
# ! НЕ учитываем, что если прил-ые встретились одинаковое количество раз, нужно взять то, которое в теме стоит ближе к началу 

def adj_in_lables(topics_dict, topic): # возвращает словарь {прил_из_темы: сколько_раз_оно_встретилось_в_метках}
    adj_in_topic = []
    for topic_word in topic.split(" "):
        if "ADJF" in morph.parse(topic_word)[0].tag: 
            adj_in_topic.append(topic_word) # получили список прилагательных в данной теме       
    adj_from_topic_in_labels = []
    for label in topics_dict[topic]:
        label_normalized = []
        words_in_label = label.split(" ")
        for a_word in words_in_label:
            label_normalized.append(morph.parse(a_word)[0].normal_form)
        for adj in adj_in_topic:
            if adj in label_normalized:
                adj_from_topic_in_labels.append(adj) # получили список прилагательных из темы в метках
    freq_dist = FreqDist(adj_from_topic_in_labels) # получили их распределение
    freq = {}
    for item in freq_dist.most_common(len(freq_dist)):
        freq[item[0]] = item[1]
    return freq
    
def two(topics_dict, topic):
    LOTS_OF = 5
    # получаем словарь, сколько раз какие существительные встречаются в метках длины >= 2
    nouns_in_big_labels = []
    for label in topics_dict[topic]:
        #if len(label.split(" ")) >= 2:
        for a_word in label.split(" "):
            if "NOUN" in morph.parse(a_word)[0].tag:
                nouns_in_big_labels.append(morph.parse(a_word)[0].normal_form)
    freq_dist = FreqDist(nouns_in_big_labels)
    freq = {}
    for item in freq_dist.most_common(len(freq_dist)):
        freq[item[0]] = item[1]
    
    # если существительное встречалолсь много раз в метках, мы удаляем все длинные метки с ним
    for noun in list(freq.keys())[::-1]: # идём с конца: если таких сущ несколько, на первом месте будет то, которого больше всего 
        if freq[noun] >= LOTS_OF:
            to_delete = []
            for label in topics_dict[topic]:
                label_normalized = []
                for a_word in label.split(" "):
                    label_normalized.append(morph.parse(a_word)[0].normal_form)
                if noun in " ".join(label_normalized): # нашли метку, в которой это слово
                    to_delete.append(label) # собираем список меток, которые нужно будет удалить
            
            for bad_label in to_delete:
                index = topics_dict[topic].index(bad_label)
                del topics_dict[topic][index] # удаляем их
                    
            # теперь, когда удалили, мы проверяем, встречаются ли в метках прилагательные из темы
            adj_from_topics_in_labels = adj_in_lables(topics_dict, topic)
            
            if len(adj_from_topics_in_labels) > 0: # если да: добавляем к нашему сущ самое частое прил и ставим на первое место
                for key in adj_from_topics_in_labels:
                    adj = key
                    break
                gender = morph.parse(noun)[0].tag.gender 
                if gender == 'masc':
                    right_adj = morph.parse(adj)[0].lexeme[0].word
                if gender == 'femn':
                    right_adj = morph.parse(adj)[0].lexeme[7].word
                if gender == 'neut':
                    right_adj = morph.parse(adj)[0].lexeme[14].word
                new_label = right_adj + " " + noun
                topics_dict[topic].insert(0, new_label)                
                
            else: # если нет, ставим существительное во мн.ч. и на первое место
                try:
                    noun_pl = morph.parse(noun)[0].lexeme[6].word
                    topics_dict[topic].insert(0, noun_pl)
                except IndexError: # нет мн ч - оставляем в единственном, приводим к словарной форме
                    topics_dict[topic].insert(0, morph.parse(a_word)[0].normal_form)
    return(topics_dict)

# ПРАВИЛО 3
def three(topics_dict, topic):
    to_move = [] # a list with pairs (label_to_be_moved, its_index)
    for label in topics_dict[topic]:
        if len(label.split(" ")) > 2:
            index = topics_dict[topic].index(label)
            to_move.append((label, index)) 
    for pair in to_move:
        new_index = topics_dict[topic].index(pair[0])
        del topics_dict[topic][new_index] # удалили все необходимые
    for pair in to_move:
        # вставляем их на две позиции позже, чем они были
        try:
            topics_dict[topic].insert(pair[1] + 2, pair[0])
        except IndexError:
            topics_dict[topic].insert(len(topics_dict[topic]), pair[0])
    return(topics_dict)

# ПРАВИЛО 4
def four(topics_dict, topic):
    # идём от последнего слова в теме к первому, так на первых позициях окажутся метки с более весомыми прилагательными
    for topic_word in topic.split(" ")[::-1]:
        if "ADJF" in morph.parse(topic_word)[0].tag: # если слово в теме - прилагательное
            for label in topics_dict[topic][::-1]: # идём от последней метки к первой - так лучшие метки окажутся сначала
                if len(label.split(" ")) > 1:
                    label_normalized = []
                    words_in_label = label.split(" ")
                    for a_word in words_in_label:
                        label_normalized.append(morph.parse(a_word)[0].normal_form)
                    if topic_word in label_normalized: # если оно же встретилось в (лемматизированной) метке
                        index = topics_dict[topic].index(label)
                        del topics_dict[topic][index] # удаляем эту метку с её позиции
                        topics_dict[topic].insert(0, " ".join(label_normalized)) # вставляем её в самое начало
                        #### !!!!! ####
    return topics_dict

In [21]:
def apply_rules(topics_dict, topic):
    four(topics_dict, topic)
    three(topics_dict, topic)
    two(topics_dict, topic)
    one(topics_dict, topic)
    return(topics_dict)

In [22]:
for topic in topics_dict:
    print(topic)
    apply_rules(topics_dict, topic)

неандерталец википедия
древний днк европа средний палеолит (неандертальцы)
() человек разумный и всевсевсе пособие
база информационный потребность
человек из денисов пещера оказаться не сапиенс и
неандерталец
кроманьонец википедия
секвенировать геном неандерталец из чагырский пещера
неандерталец такой же человек как мы просто они не
тот в с мосина южный урал в каменный век
генетика завершить работа антрополог над сам
генетический след ведущий к
чёрный дыра википедия
словарь термин
астрономия
рпастрономия класс мбоу сош №
сверхмассивный чёрный дыра википедия
млечный путь википедия
астрономия контрольный и самостоятельный работа
рабочий учебный программа по астрономия для класс
рабочий программа учебный курс по астрономия для
программа астрономия рушинскай школа №
класс рабочий программа астрономия
внеатмосферный астрономия
строение и эволюция вселенная
проект
астрономия базовый уровень класс» учебный главный
о возможность существование чёрный дыра в рамка
астрономия класс (вм чаругин)
м

In [23]:
for topic in topics_dict:
    print(topic + ' ---- ' + topics_dict[topic][0])

неандерталец википедия ---- днк
древний днк европа средний палеолит (неандертальцы) ---- цивилизация
() человек разумный и всевсевсе пособие ---- неандерталец
база информационный потребность ---- –
человек из денисов пещера оказаться не сапиенс и ---- ктоть
неандерталец ---- эпигенетический
кроманьонец википедия ---- неандерталец
секвенировать геном неандерталец из чагырский пещера ---- неандерталец
неандерталец такой же человек как мы просто они не ---- (неандерталец)
тот в с мосина южный урал в каменный век ---- чеплыгин
генетика завершить работа антрополог над сам ---- эволюция
генетический след ведущий к ---- год
чёрный дыра википедия ---- галактический
словарь термин ---- космический
астрономия ---- спектррг
рпастрономия класс мбоу сош № ---- многоканальный
сверхмассивный чёрный дыра википедия ---- астроном
млечный путь википедия ---- галактический
астрономия контрольный и самостоятельный работа ---- общий
рабочий учебный программа по астрономия для класс ---- астрономиякласс
рабо

In [24]:
import codecs

path = "lias_topics/GLDA_googled_Lyasdata10.txt"
first_n = 10
with codecs.open(path, 'w', 'utf8') as output_file:
    for topic in topics_dict:
        output_file.write('TOPIC:' + ", ".join(topics_dict[topic][:1]) + '\n')
        output_file.write('LABELS:  ' + topic + '\n\n')